In [1]:
import os
import torch
import matplotlib.pyplot as plt
from skimage import io, transform
from torchvision import transforms, utils
import numpy as np
import nibabel as nib
from random import randint
from PIL import Image

### I. Data loading
The dataset containing a subset of the Human Connectome Project data. 
It contains a total of 285 participants and a total of 330 cases.

The images for one case are stored in its own specific folder, i.e. sub-XX where XX is the HCP participant labell. Note that 45 of them (distinguished in their name by *-V2) are follow-up scans of the same participant. Thus, if you want to consider only one case per participant, you should discard the *-V2s.

- sub-XX_T1w.nii.gz which is the original T1w image (input of the system)
- sub-XX_T1w._brainmask.nii.gz which is the brainmask used for the generation of the 5 different scales of parcellation
- sub-XX_class-CSF_T1w and sub_class-WM_T1w are masks of the CSF/ White Matter tissues estimated by Freesurfer during the parcellation pipeline..
- sub-XX_parc_scaleZ which is the parcellation (labels) for scale Z. scale1 corresponds to the 83 labels generated by freesurfer. scale2->5 correspond to parcellations at a finer scale. I guess we could start using scale1 as the output of the system.

In [2]:
sub_dir = '/home/xiaoyu/ds-hcp/T1_image'
parc5_dir = '/home/xiaoyu/ds-hcp/parc_5'

In [3]:
sub_list = os.listdir(sub_dir)
print('Subject num:')
print(len(sub_list))
parc5_list = os.listdir(parc5_dir)
print('\nParc5 num:')
print(len(parc5_list))

Subject num:
330

Parc5 num:
330


### II. Slice the 3D MRI T1w data
There are 330 3D MRI T1w data, for each T1w data, there will be 3 planes, i.e. Axial plane, Coronal plane and Sagittal plane.

In [4]:
# iterate over 330 T1w data
for i in range(330):
    sub_str = sub_list[i]
    sub_nifti = nib.load(os.path.join(sub_dir,sub_str))
    sub_arr = sub_nifti.get_fdata()
    sub_tensor = torch.from_numpy(sub_arr)
    # get the Axial plane. Get 182 images for each T1w data, image size is 182x217
    for j in range(182):
        axial_slice = sub_tensor[:,:,j]
        axial_arr = axial_slice.byte().numpy()
        axial_im = Image.fromarray(axial_arr)
        axial_im.save('/home/xiaoyu/MRIdata/T1w/axial/sub{}/slice_{}.jpg'.format(i,j))
    for k in range(217):
        sagittal_slice = sub_tensor[:,k,:]
        sagittal_arr = sagittal_slice.byte().numpy()
        sagittal_im = Image.fromarray(sagittal_arr)
        sagittal_im.save('/home/xiaoyu/MRIdata/T1w/sagittal/sub{}/slice_{}.jpg'.format(i,k))
    for m in range(182):
        coronal_slice = sub_tensor[m,:,:]
        coronal_arr = coronal_slice.byte().numpy()
        coronal_im = Image.fromarray(coronal_arr)
        coronal_im.save('/home/xiaoyu/MRIdata/T1w/coronal/sub{}/slice_{}.jpg'.format(i,m))

### III. Slice the 3D parcellation scale 5 data
There are 330 3D MRI parcellation scale-5 data. For each parc_5 data, there will be 3 planes, the slice is the same for T1w data

In [5]:
# iterate over 330 parcellation scale-5 data
for i in range(330):
    parc5_str = parc5_list[i]
    parc5_nifti = nib.load(os.path.join(parc5_dir,parc5_str))
    parc5_arr = parc5_nifti.get_fdata()
    parc5_tensor = torch.from_numpy(parc5_arr)

    for j in range(182):
        axial_slice = parc5_tensor[:,:,j]
        axial_arr = axial_slice.byte().numpy()
        axial_im = Image.fromarray(axial_arr)
        axial_im.save('/home/xiaoyu/MRIdata/parc_5/axial/sub{}/slice_{}.jpg'.format(i,j))
    for k in range(217):
        sagittal_slice = parc5_tensor[:,k,:]
        sagittal_arr = sagittal_slice.byte().numpy()
        sagittal_im = Image.fromarray(sagittal_arr)
        sagittal_im.save('/home/xiaoyu/MRIdata/parc_5/sagittal/sub{}/slice_{}.jpg'.format(i,k))
    for m in range(182):
        coronal_slice = parc5_tensor[m,:,:]
        coronal_arr = coronal_slice.byte().numpy()
        coronal_im = Image.fromarray(coronal_arr)
        coronal_im.save('/home/xiaoyu/MRIdata/parc_5/coronal/sub{}/slice_{}.jpg'.format(i,m))
